In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from sportsipy.ncaab import boxscore
from sportsipy.ncaab.schedule import Schedule
from sportsipy.ncaab.teams import Teams
from sportsipy.ncaab.teams import Team

In [2]:
# Set counter to watch progress
cnt = 0
# Loop through all teams in database (353 teams)
for team in Teams("2020"):
    cnt += 1
    # 3 Teams caused an error so do try/except so the loop doesnt stop
    try:
        print(cnt)
# Returns a Pandas DataFrame of all metrics for all game Boxscores for a season
        df = team.schedule.dataframe_extended
        
        # add team name to dataframe
        df['team_name'] = team.name
        
        
        # add conference name to dataframe
        df['conf'] = team.conference
                
        # send to csv
        df.to_csv(f'../data/team_csvs/2019-2020/{team.name}_season_boxscores_2019.csv')
    except:
        print(cnt,'did not work')

1
2
3
4
5
6
7
8
9
10
11
12
12 did not work
13
13 did not work
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
46 did not work
47
48
49
50
50 did not work
51
52
53
53 did not work
54
55
56
57
58
59
60
61
62
63
64
65
65 did not work
66
67
68
69
70
70 did not work
71
72
72 did not work
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
109 did not work
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
155 did not work
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
23

In [9]:
big_df = pd.DataFrame()
season = Teams(year='2020')
for team in season:
    team_df = team.dataframe
    big_df = pd.concat([big_df, team_df], axis=0)
     

In [ ]:
# First, concat all the boxscore files by team into one massive one
# This loops through all files ending in .csv and concatenates them into a single df
from glob import glob
# grab all filenames ending with .csv
#TODO: This will be where you create the trimmed.csv - use the backup files. 
filenames = glob('../data/team_csvs/2019-2020 copy/*.csv')
# loop through all file names adding the contents to one dataframe
ncaa_boxscore_df = pd.concat([pd.read_csv(f) for f in filenames], ignore_index = True)

ncaa_boxscore_df.to_csv('../data/team_csvs/2019-2020/2020_ncaa_season_boxscores.csv')

In [10]:
big_df.to_csv('../data/team_csvs/2019-2020/teams.csv')

In [11]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 353 entries, ABILENE-CHRISTIAN to YOUNGSTOWN-STATE
Data columns (total 84 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   abbreviation                            353 non-null    object 
 1   assist_percentage                       353 non-null    float64
 2   assists                                 353 non-null    int64  
 3   away_losses                             353 non-null    int64  
 4   away_wins                               353 non-null    int64  
 5   block_percentage                        353 non-null    float64
 6   blocks                                  353 non-null    int64  
 7   conference                              353 non-null    object 
 8   conference_losses                       353 non-null    int64  
 9   conference_wins                         353 non-null    int64  
 10  defensive_rebounds                    

In [12]:
# TODO: this will be where we loop through all files and get the CSV as a new dataframe
directory = '../data/team_csvs/2019-2020/'
for filename in os.scandir(directory):
    # create DF from CSV 
    try: 
        teamschedule_df = pd.read_csv(filename)

        #filename variable is an nt.DirEntry type. We need to convert it to string
        file_name = str(filename)
        print(file_name)
        # <DirEntry '109_Georgia_season_boxscores_2019.csv'>
        csvname = file_name[11:-2]
        print(csvname)

        # Now, we need to slice the string in a sustainable way. We'll slice out everything up to and including the first underscore
        file_name = file_name.split('_', 1)
        print(file_name)
        # ["<DirEntry '109", "Georgia_season_boxscores_2019.csv'>"]

        team_name = file_name[0]
        file_name = file_name[1]
        print(file_name)
        # Georgia_season_boxscores_2019.csv'>

        #next we extract team name and season - we know that everything after the name is constant for all files
        team_name = team_name[11:]
        season = file_name[-10:-6]
        print(team_name)
        print(season)
        print('getting team abbreviation')

        temp_df = big_df.loc[big_df['name'] == team_name]
        abbrev = temp_df.iloc[0]['abbreviation']


        #for team in big_df[]:
        #    if big_df['name'] == team_name:
        #        abbrev = big_df['abbreviation']
        #        break
            
        print (abbrev)
        teamschedule_df.rename(columns={'Unnamed: 0': 'boxscore'}, inplace=True)
        teamschedule_df['game_number'] = teamschedule_df.index

        # Conditional column - we have a pair of criteria that tell us if a given game is home based on the winner of the game (home vs. away) and 
        # the abbreviation of the winning team
        teamschedule_df['{}_homegame'.format(abbrev)] = (teamschedule_df['winner'] == 'Home') & (teamschedule_df['winning_abbr'] == abbrev)
        teamschedule_df['{}_homegame2'.format(abbrev)] = (teamschedule_df['winner'] == 'Away') & (teamschedule_df['winning_abbr'] != abbrev)

        # set team_abbrev_home to True if either of the home game criteria that we ran above are true
        teamschedule_df['{}_home'.format(abbrev)] = (teamschedule_df['{}_homegame'.format(abbrev)] == True) | (teamschedule_df['{}_homegame2'.format(abbrev)] == True)

        # Make the game location (from perspective of the team) equal to Home if the team_abbrev_home column is True, otherwise make it away
        teamschedule_df['{}_location'.format(abbrev)] = teamschedule_df.apply(lambda x: 'Home' if x['{}_home'.format(abbrev)] == True else 'Away', axis=1)

        # Create a column with True/False based on whether the team won their game. This will be used later to calc win percentage
        teamschedule_df['team_win_bool'] = (teamschedule_df['winning_abbr'] == abbrev)

        # Set each game stat equal to the home team stat if they played at home, otherwise set to the away team stat
        teamschedule_df['total_threes_made'] = teamschedule_df.apply(lambda x: x['home_three_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_three_point_field_goals'], axis=1)
        teamschedule_df['total_2pt_made'] = teamschedule_df.apply(lambda x: x['home_two_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_two_point_field_goals'], axis=1)
        teamschedule_df['total_fg_attempts'] = teamschedule_df.apply(lambda x: x['home_field_goal_attempts'] if x['{}_home'.format(abbrev)] == True else x['away_field_goal_attempts'], axis=1)

        # Do the same thing for rebounds instead of shooting
        # rebounding percentage = team rebounds / total rebounds
        teamschedule_df['team_rebounds'] = teamschedule_df.apply(lambda x: x['home_total_rebounds'] if x['{}_home'.format(abbrev)] == True else x['away_total_rebounds'], axis=1)
        teamschedule_df['total_rebounds'] = teamschedule_df['home_total_rebounds'] + teamschedule_df['away_total_rebounds']

        # Do the same thing for defensive statistics - remember we need to invert this
        teamschedule_df['total_opp_threes_made'] = teamschedule_df.apply(lambda x: x['away_three_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['home_three_point_field_goals'], axis=1)
        teamschedule_df['total_opp_2pt_made'] = teamschedule_df.apply(lambda x: x['away_two_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['home_two_point_field_goals'], axis=1)
        teamschedule_df['total_opp_fg_attempts'] = teamschedule_df.apply(lambda x: x['away_field_goal_attempts'] if x['{}_home'.format(abbrev)] == True else x['home_field_goal_attempts'], axis=1)

        #finally, calculate the rolling win percentage
        # adding 1 to game number so that we can easily calculate the wins and losses
        teamschedule_df['team_wins'] = 0
        teamschedule_df['team_losses'] = 0
        teamschedule_df['team_wins'] = teamschedule_df.apply(lambda x: x['team_wins'] + 1 if x['team_win_bool'] == True else x['team_wins'], axis = 1)
        teamschedule_df['team_losses'] = teamschedule_df.apply(lambda x: x['team_losses'] + 1 if x['team_win_bool'] == False else x['team_losses'], axis = 1)

        #calculate the cumulative pregame stats heading in to each game
        teamschedule_df['pregame_total_threes_made'] = teamschedule_df.total_threes_made.cumsum() - teamschedule_df.total_threes_made
        teamschedule_df['pregame_total_2pt_made'] = teamschedule_df.total_2pt_made.cumsum() - teamschedule_df.total_2pt_made
        teamschedule_df['pregame_total_fg_attempts'] = teamschedule_df.total_fg_attempts.cumsum() - teamschedule_df.total_fg_attempts

        #calculate the actual target statistic which is eFG 
        teamschedule_df['pregame_eFG'] = (teamschedule_df.pregame_total_2pt_made + (teamschedule_df.pregame_total_threes_made * 1.5)) / teamschedule_df.pregame_total_fg_attempts

        #calculate the cumulative rebound percentage
        teamschedule_df['pregame_team_rebounds'] = teamschedule_df.team_rebounds.cumsum() - teamschedule_df.team_rebounds
        teamschedule_df['pregame_total_rebounds'] = teamschedule_df.total_rebounds.cumsum() - teamschedule_df.total_rebounds
        teamschedule_df['pregame_rebound_percentage'] = teamschedule_df.pregame_team_rebounds / teamschedule_df.pregame_total_rebounds


        #calculate the cumulative defensive statistic
        teamschedule_df['pregame_total_opp_threes_made'] = teamschedule_df.total_opp_threes_made.cumsum() - teamschedule_df.total_opp_threes_made
        teamschedule_df['pregame_total_opp_2pt_made'] = teamschedule_df.total_opp_2pt_made.cumsum() - teamschedule_df.total_opp_2pt_made
        teamschedule_df['pregame_total_opp_fg_attempts'] = teamschedule_df.total_opp_fg_attempts.cumsum() - teamschedule_df.total_opp_fg_attempts

        # calculate the defensive eFG - aka opponents eFG
        teamschedule_df['pregame_eFG_allowed'] = (teamschedule_df.pregame_total_opp_2pt_made + (teamschedule_df.pregame_total_opp_threes_made * 1.5)) / teamschedule_df.pregame_total_opp_fg_attempts

        #calculate the pregame win percentage - it's the total wins after the game, minus result of game (loss would have no impact on total wins)
        # minus total losses after game (win would have no impact) divided by the sum of the two
        teamschedule_df['pregame_win_percentage'] = (teamschedule_df.team_wins.cumsum() - teamschedule_df.team_wins) / ((teamschedule_df.team_wins.cumsum() - teamschedule_df.team_wins) + (teamschedule_df.team_losses.cumsum() - teamschedule_df.team_losses))



        print('dataframe complete')

        # Overwrite the CSV with the new dataframe containing stats
        teamschedule_df.to_csv('../data/team_csvs/2019-2020/{}'.format(csvname))
        print(team_name, ' CSV complete')
    except:
        print(team_name, 'error')



<DirEntry 'Abilene Christian_season_boxscores_2019.csv'>
Abilene Christian_season_boxscores_2019.csv
["<DirEntry 'Abilene Christian", "season_boxscores_2019.csv'>"]
season_boxscores_2019.csv'>
Abilene Christian
2019
getting team abbreviation
ABILENE-CHRISTIAN
dataframe complete
Abilene Christian  CSV complete
<DirEntry 'Air Force_season_boxscores_2019.csv'>
Air Force_season_boxscores_2019.csv
["<DirEntry 'Air Force", "season_boxscores_2019.csv'>"]
season_boxscores_2019.csv'>
Air Force
2019
getting team abbreviation
AIR-FORCE
dataframe complete
Air Force  CSV complete
<DirEntry 'Akron_season_boxscores_2019.csv'>
Akron_season_boxscores_2019.csv
["<DirEntry 'Akron", "season_boxscores_2019.csv'>"]
season_boxscores_2019.csv'>
Akron
2019
getting team abbreviation
AKRON
dataframe complete
Akron  CSV complete
<DirEntry 'Alabama A&M_season_boxscores_2019.csv'>
Alabama A&M_season_boxscores_2019.csv
["<DirEntry 'Alabama A&M", "season_boxscores_2019.csv'>"]
season_boxscores_2019.csv'>
Alabama A&M


In [ ]:
#All of the below cell is to play around
#directory = '../data/team_csvs/2019-2020/'
#filename = '283_South Florida_season_boxscores_2019.csv'
#teamschedule_df = pd.read_csv(directory + filename)

#filename variable is an nt.DirEntry type. We need to convert it to string
#file_name = str(filename)
#print(file_name)

# Now, we need to slice the string in a sustainable way. We'll slice out everything up to and including the first underscore
#file_name = file_name.split('_', 1)
#print(file_name)

#file_name = file_name[1]
#print(file_name)
#next we extract team name and season - we know that everything after the name is constant for all files
#team_name = file_name[:-26]
#season = file_name[-8:-4]
#print(team_name)
#print(season)
#print('getting team abbreviation')

#for team in Teams(season):
#    if team.name == team_name:
#        abbrev = team.abbreviation
#        break

#print (abbrev)
#teamschedule_df.rename(columns={'Unnamed: 0': 'boxscore'}, inplace=True)
#teamschedule_df['game_number'] = teamschedule_df.index

# Conditional column - we have a pair of criteria that tell us if a given game is home based on the winner of the game (home vs. away) and 
# the abbreviation of the winning team
#teamschedule_df['{}_homegame'.format(abbrev)] = (teamschedule_df['winner'] == 'Home') & (teamschedule_df['winning_abbr'] == abbrev)
#teamschedule_df['{}_homegame2'.format(abbrev)] = (teamschedule_df['winner'] == 'Away') & (teamschedule_df['winning_abbr'] != abbrev)

# set team_abbrev_home to True if either of the home game criteria that we ran above are true
#teamschedule_df['{}_home'.format(abbrev)] = (teamschedule_df['{}_homegame'.format(abbrev)] == True) | (teamschedule_df['{}_homegame2'.format(abbrev)] == True)

# Make the game location (from perspective of the team) equal to Home if the team_abbrev_home column is True, otherwise make it away
#teamschedule_df['{}_location'.format(abbrev)] = teamschedule_df.apply(lambda x: 'Home' if x['{}_home'.format(abbrev)] == True else 'Away', axis=1)

# Set each game stat equal to the home team stat if they played at home, otherwise set to the away team stat
#teamschedule_df['total_threes_made'] = teamschedule_df.apply(lambda x: x['home_three_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_three_point_field_goals'], axis=1)
#teamschedule_df['total_fg_made'] = teamschedule_df.apply(lambda x: x['home_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_field_goals'], axis=1)
#teamschedule_df['total_fg_attempts'] = teamschedule_df.apply(lambda x: x['home_field_goal_attempts'] if x['{}_home'.format(abbrev)] == True else x['away_field_goal_attempts'], axis=1)

#calculate the cumulative pregame stats heading in to each game
#teamschedule_df['pregame_total_threes_made'] = teamschedule_df.total_threes_made.cumsum() - teamschedule_df.total_threes_made
#teamschedule_df['pregame_total_fg_made'] = teamschedule_df.total_fg_made.cumsum() - teamschedule_df.total_fg_made
#teamschedule_df['pregame_total_fg_attempts'] = teamschedule_df.total_fg_attempts.cumsum() - teamschedule_df.total_fg_attempts

#calculate the actual target statistic which is eFG 
#teamschedule_df['pregame_eFG'] = (teamschedule_df.pregame_total_fg_made + (teamschedule_df.pregame_total_threes_made / 2)) / teamschedule_df.pregame_total_fg_attempts
# (2PM + .5 * 3PM) / FGA
#print('dataframe complete')
#teamschedule_df.to_csv('../data/team_csvs/2019-2020/{}'.format(filename))
#print(team_name, ' CSV complete')



In [ ]:
teamschedule_df.head()

In [ ]:
teamschedule_df.head()

In [ ]:
teamschedule_df.head()

In [ ]:



#each team's csv should have that team's rolling stats in the CSV. Then, for each game in the giant boxscore DF, 
# I can go grab the team's pregame eFG for that particular game number. Would prob have to index on boxscore uri
